In [185]:
import requests

from ape import networks, Contract
import pandas as pd

from helpers.coingecko import get_prices
from helpers.subgraph_endpoints import get_cost_per_vote_after_fee, get_proposal_info, get_tvl_balancer_badger_pool, get_tvl_balancer_digg_pool
from helpers.aura import get_gravi_in_balancer_pool, weekly_emissions_after_fee, aura_mint_ratio
from helpers.vp_info import get_council_vp_fee, get_voter_vp
from helpers.bribe_info import get_bribe_info, MAX_BRIBE, GAUGE_CAP
from helpers.formatter import pct_format, dollar_format

In [186]:
networks.parse_network_choice("ethereum:mainnet").__enter__()

INFO: Connecting to existing Geth node at 'https://eth-mainnet.g.alchemy.com/v2/Ct5gz9eC4qAPrNN4d-6l9CrOHePE5IVH'.


<geth chain_id=1>

In [187]:
bal_price, aura_price,  badger_price = get_prices()
data = [["${:.2f}".format(badger_price),  "${:.2f}".format(bal_price), "${:.2f}".format(aura_price)]]
headers = ["Badger price", "Balancer price", "Aura price"]
df_prices = pd.DataFrame(data, columns=headers)
df_prices

,Badger price,Balancer price,Aura price
0,$3.41,$4.99,$1.76


In [188]:
# NOTE: modify with right proposal (latest & current)
proposal_id_last = "0x50f8e76b64188af61aec0fe6a3877f0a16bb683490acefd2ea0359e166098b96"
proposal_id_current = "0x50f8e76b64188af61aec0fe6a3877f0a16bb683490acefd2ea0359e166098b96"

cost_per_vote_after_fee = get_cost_per_vote_after_fee()

# check active voter in past proposal as ref
response_last_proposal = get_proposal_info(proposal_id_last)
response_current_proposal = get_proposal_info(proposal_id_current)

block_current_proposal = int(response_current_proposal["snapshot"])
aura_voting_actively_last_round = response_last_proposal["scores_total"]
data = [[block_current_proposal,  f'{aura_voting_actively_last_round:.0f}', dollar_format(cost_per_vote_after_fee)]]
headers = ["Proposal block height", "Past Round Active vlAURA votes", "Cost per Vote Last Round"]
df_llama = pd.DataFrame(data, columns=headers)
df_llama


,Proposal block height,Past Round Active vlAURA votes,Cost per Vote Last Round
0,15635887,6830592,$0.0652


In [189]:
# emissions ecosystem
aura = Contract('0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF')
aura_mint_ratio = aura_mint_ratio(aura, block_current_proposal)
weekly_emissions_after_fee = weekly_emissions_after_fee(aura_mint_ratio, bal_price, aura_price)
biweekly_emissions = weekly_emissions_after_fee * 2
data = [[aura_mint_ratio, dollar_format(biweekly_emissions)]]
headers = ["Aura Mint Balancer Ratio", "Bi-weekly Emissions"]
df_emissions = pd.DataFrame(data, columns=headers)
df_emissions


,Aura Mint Balancer Ratio,Bi-weekly Emissions
0,3.752714,$2521863.7828


In [190]:
vlAURA = Contract("0x3Fa73f1E5d8A792C80F426fc8F84FBF7Ce9bBCAC")
vlAURA_ts = vlAURA.totalSupply(block_identifier=block_current_proposal)/1e18
council_fee = get_council_vp_fee(vlAURA, block_current_proposal)

# digg pool
balancer_vault = Contract("0xBA12222222228d8Ba445958a75a0704d566BF2C8")
gravi_in_digg_pool = get_gravi_in_balancer_pool(balancer_vault, block_current_proposal)
total_gravi_vp_for_digg = gravi_in_digg_pool + council_fee

treasury_votes = get_voter_vp(vlAURA, block_current_proposal)
total_vp_badgerdao = treasury_votes + council_fee

pct_controlled_aura_by_badger = total_vp_badgerdao/ vlAURA_ts
pct_controlled_aura_by_badger_active = total_vp_badgerdao/ aura_voting_actively_last_round
pct_controlled_aura_by_council_active = total_gravi_vp_for_digg / aura_voting_actively_last_round
data = [[treasury_votes, council_fee, total_vp_badgerdao, gravi_in_digg_pool, total_gravi_vp_for_digg, pct_format(pct_controlled_aura_by_badger), pct_format(pct_controlled_aura_by_badger_active)]]
headers = ["Treasury VP", "Council Fee", "Total Badger VP", "Gravi in Digg pool", "Total Badger Council VP - if voting for DIGG", "Badger Aura Controlled - everyone Votes", "Badger Aura Controlled - proactive voters"]
df_vp = pd.DataFrame(data, columns=headers)
df_vp

,Treasury VP,Council Fee,Total Badger VP,Gravi in Digg pool,Total Badger Council VP - if voting for DIGG,Badger Aura Controlled - everyone Votes,Badger Aura Controlled - proactive voters
0,182298.367156,91833.486207,274131.853364,256854.746895,348688.233102,3.396%,4.013%


In [191]:
vebal = Contract("0xC128a9954e6c874eA3d62ce62B468bA073093F25")
vebal_ts = vebal.totalSupplyAt(block_current_proposal) / 1e18
vebal_aura_controlled = vebal.balanceOfAt("0xaF52695E1bB01A16D33D7194C28C42b10e0Dbec2", block_current_proposal)/1e18
pct_aura_vebal_controlled = vebal_aura_controlled/vebal_ts
pct_badger_vebal_controlled = pct_controlled_aura_by_badger*pct_aura_vebal_controlled
pct_badger_vebal_controlled_active = pct_controlled_aura_by_badger_active*pct_aura_vebal_controlled
pct_council_vebal_controlled_active = pct_controlled_aura_by_council_active * pct_aura_vebal_controlled
data = [[pct_format(pct_aura_vebal_controlled), pct_format(pct_badger_vebal_controlled), pct_format(pct_badger_vebal_controlled_active)]]
headers = ["Aura veBAL controlled", "Badger veBAL controlled - everyone votes", "Badger veBAL controlled - proactive voters"]
df_vebal = pd.DataFrame(data, columns=headers)
df_vebal

,Aura veBAL controlled,Badger veBAL controlled - everyone votes,Badger veBAL controlled - proactive voters
0,25.450%,0.864%,1.021%


In [192]:
# NOTE: block to max out badger/wbtc cap
vlAURA_req, cost_of_max_cap, badger_to_bribe_badger_pool = get_bribe_info(total_vp_badgerdao, pct_badger_vebal_controlled_active, cost_per_vote_after_fee,badger_price)
bribe_reclycing_badger_voting = badger_to_bribe_badger_pool * (total_vp_badgerdao/vlAURA_req)
data = [[vlAURA_req, dollar_format(cost_of_max_cap), badger_to_bribe_badger_pool, bribe_reclycing_badger_voting]]
headers = ["vlAURA Required", "Cost of Maxing Cap", "Badger Bribe Max Cap", "Expected Bribe reclyced"]
df_bribes_badger_pool = pd.DataFrame(data, columns=headers)
df_bribes_badger_pool

,vlAURA Required,Cost of Maxing Cap,Badger Bribe Max Cap,Expected Bribe reclyced
0,536779.36656,$34992.7857,10261.813989,5240.68223


In [193]:
# NOTE: block to max out digg/wbtc/gravi cap
vlAURA_req, cost_of_max_cap, badger_to_bribe_digg_pool = get_bribe_info(total_vp_badgerdao, pct_council_vebal_controlled_active, cost_per_vote_after_fee, badger_price)
bribe_reclycing_digg_voting = badger_to_bribe_digg_pool * (total_gravi_vp_for_digg/vlAURA_req)
data = [[vlAURA_req, dollar_format(cost_of_max_cap), badger_to_bribe_digg_pool, bribe_reclycing_digg_voting]]
headers = ["vlAURA Required", "Cost of Maxing Cap", "Badger Bribe Max Cap", "Expected Bribe reclyced"]
df_bribes_digg_pool = pd.DataFrame(data, columns=headers)
df_bribes_digg_pool

,vlAURA Required,Cost of Maxing Cap,Badger Bribe Max Cap,Expected Bribe reclyced
0,422005.415248,$27510.6421,8067.636991,6666.004715


In [194]:
# amount of bribe based on allowance free for bribing elsewhere
bribe_overflow_badger_pool = MAX_BRIBE - badger_to_bribe_badger_pool
bribe_overflow_digg_pool = MAX_BRIBE - badger_to_bribe_digg_pool
data = [[bribe_overflow_badger_pool, bribe_overflow_digg_pool]]
headers = ["Badger Bribe Overflow - Bribing badger/wbtc pool", "Badger Bribe Overflow - Bribing digg/wbtc/graviaura pool"]
df_bribes_overflow = pd.DataFrame(data, columns=headers)
df_bribes_overflow


,Badger Bribe Overflow - Bribing badger/wbtc pool,Badger Bribe Overflow - Bribing digg/wbtc/graviaura pool
0,5738.186011,7932.363009


In [195]:
# treasury expected earnings bi-weekly expectation post-bribe into BADGER pool
tvl_badger = get_tvl_balancer_badger_pool()
badger_pool = Contract("0xb460DAa847c45f1C4a41cb05BFB3b51c92e41B36")
pool_supply = badger_pool.totalSupply() / 1e18
reward_badger_aura = Contract("0x05df1E87f41F793D9e03d341Cdc315b76595654C")
vault_rewards_contract_balance = reward_badger_aura.balanceOf("0xD0A7A8B98957b9CD3cFB9c0425AbE44551158e9e", block_identifier=block_current_proposal)/1e18
vault_owned_tcl = tvl_badger * (vault_rewards_contract_balance/pool_supply)
pool_capture = vault_owned_tcl / tvl_badger
pool_earnings = biweekly_emissions * GAUGE_CAP
real_bribe_aura_expense = (badger_to_bribe_badger_pool - bribe_reclycing_badger_voting) * badger_price
treasury_roi = (pool_earnings*pool_capture) / real_bribe_aura_expense
data = [[dollar_format(pool_earnings), dollar_format(tvl_badger), pct_format(pool_capture), dollar_format(pool_earnings*pool_capture), dollar_format(real_bribe_aura_expense), pct_format(treasury_roi)]]
headers = ["Pool yield (USD) - biweekly", "BADGER Pool TVL", "BADGER Pool Capture", "Treasury yield earned (USD) - biweekly", "Bribe Aura Real Cost", "Treasury ROI - biweekly"]
df_earnings_badger_pool = pd.DataFrame(data, columns=headers)
df_earnings_badger_pool

,Pool yield (USD) - biweekly,BADGER Pool TVL,BADGER Pool Capture,Treasury yield earned (USD) - biweekly,Bribe Aura Real Cost,Treasury ROI - biweekly
0,$50437.2757,$7006378.9770,39.150%,$19746.0004,$17122.0593,115.325%


In [196]:
# treasury expected earnings bi-weekly expectation post-bribe into DIGG pool
tvl_digg = get_tvl_balancer_digg_pool()
digg_pool = Contract("0x8eB6c82C3081bBBd45DcAC5afA631aaC53478b7C")
pool_supply = digg_pool.totalSupply() / 1e18
reward_badger_aura = Contract("0xe86f0312b06126855810b4a13a43c3e2b1b8dd90")
vault_rewards_contract_balance = reward_badger_aura.balanceOf("0xD0A7A8B98957b9CD3cFB9c0425AbE44551158e9e", block_identifier=block_current_proposal)/1e18
vault_owned_tcl = tvl_digg * (vault_rewards_contract_balance/pool_supply)
pool_capture = vault_owned_tcl / tvl_digg
pool_earnings = biweekly_emissions * GAUGE_CAP
real_bribe_aura_expense = (badger_to_bribe_digg_pool - bribe_reclycing_digg_voting) * badger_price
# NOTE: remind that in the strategy `bribesRedirectionPath[badger]=trops` all badger claimed goes back to treasury
treasury_roi = (pool_earnings*pool_capture) / real_bribe_aura_expense
data = [[dollar_format(pool_earnings), dollar_format(tvl_digg), pct_format(pool_capture), dollar_format(pool_earnings*pool_capture), dollar_format(real_bribe_aura_expense), pct_format(treasury_roi)]]
headers = ["Pool yield (USD) - biweekly", "DIGG Pool TVL", "DIGG Pool Capture", "Treasury yield earned (USD) - biweekly", "Bribe Aura Real Cost", "Treasury ROI - biweekly"]
df_earnings_badger_pool = pd.DataFrame(data, columns=headers)
df_earnings_badger_pool

,Pool yield (USD) - biweekly,DIGG Pool TVL,DIGG Pool Capture,Treasury yield earned (USD) - biweekly,Bribe Aura Real Cost,Treasury ROI - biweekly
0,$50437.2757,$2353800.8806,35.832%,$18072.6727,$4779.5661,378.124%
